In [1]:
import dataclasses
import znflow
import random
import time

In [2]:
@dataclasses.dataclass
class Node(znflow.Node):
    inputs: float
    outputs: float = None
    
    def run(self):
        time.sleep(0.1)
        self.outputs = self.inputs * 2

@dataclasses.dataclass
class SumNodes(znflow.Node):
    inputs: float
    outputs: float = None

    def run(self):
        time.sleep(0.1)
        self.outputs = sum(self.inputs)

In [3]:
k = 5
j = 5
i = 5

with znflow.DiGraph() as graph:
    kdx_nodes = []
    for kdx in range(k):
        jdx_nodes = []
        for jdx in range(j):
            idx_nodes = []
            for idx in range(i):
                idx_nodes.append(Node(inputs=random.random()))
            jdx_nodes.append(SumNodes(inputs=[x.outputs for x in idx_nodes]))
        kdx_nodes.append(SumNodes(inputs=[x.outputs for x in jdx_nodes]))
            
    end_node = SumNodes(inputs=[x.outputs for x in kdx_nodes])

znflow.draw(graph)

%%time
graph.run()

In [4]:
from dask.distributed import Client

In [5]:
client = Client()

/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39227 instead
  warnings.warn(


In [6]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39227/status,
Dashboard: http://127.0.0.1:39227/status,Workers: 5
Total threads: 20,Total memory: 62.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44231,Workers: 5
Dashboard: http://127.0.0.1:39227/status,Total threads: 20
Started: Just now,Total memory: 62.60 GiB
Comm: tcp://127.0.0.1:39837,Total threads: 4
Dashboard: http://127.0.0.1:43387/status,Memory: 12.52 GiB
Nanny: tcp://127.0.0.1:43587,


In [7]:
deployment = znflow.deployment.Deployment(graph=graph, client=client)

In [8]:
deployment.submit_graph()

2023-02-10 11:33:52,119 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/core.py", line 158, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/core.py", line 138, in _decode_default
    return merge_and_deserialize(
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 497, in merge_and_deserialize
    return deserialize(header, merged_frames, deserializers=deserializers)
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 426, in deserialize
    return loads(header, frames)
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/serialize.p

In [ ]:
%%time
# results = deployment.get_results(graph.nodes)
print(deployment.get_results(end_node))

2023-02-10 11:33:52,562 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/core.py", line 158, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/core.py", line 138, in _decode_default
    return merge_and_deserialize(
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 497, in merge_and_deserialize
    return deserialize(header, merged_frames, deserializers=deserializers)
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/serialize.py", line 426, in deserialize
    return loads(header, frames)
  File "/data/fzills/miniconda3/envs/znflow/lib/python3.10/site-packages/distributed/protocol/serialize.p

In [ ]:
%%time
graph.run()